# Resolvendo o XOR com Três Perceptrons

Este notebook mostra como resolver o problema lógico do XOR utilizando três perceptrons simples, sem usar redes multicamadas com retropropagação.

A lógica usada é baseada na decomposição do XOR:

$$XOR(x_1, x_2) = (x_1 \lor x_2) \land \neg(x_1 \land x_2)$$

Vamos treinar três Perceptrons para implementar essa lógica combinada.

## Arquitetura com 3 Perceptrons

```
 x1 ---->┐
         ├─> [P1: OR] ─┐
 x2 ---->┘             │
                       ├─> [P3: XOR logic] ──> Saída
 x1 ---->┐             │
         ├─> [P2: AND] ┘
 x2 ---->┘
```

- **P1:** aprende OR
- **P2:** aprende AND
- **P3:** recebe as saídas e fazer: `OR AND NOT(AND)`

## Tabela de Verdade XOR com Intermediários

| x1 | x2 | OR | AND | NOT(AND) | OR AND NOT(AND) = XOR |
|----|----|----|-----|-----------|------------------------|
| 0  | 0  | 0  |  0  |     1     |           0            |
| 0  | 1  | 1  |  0  |     1     |           1            |
| 1  | 0  | 1  |  0  |     1     |           1            |
| 1  | 1  | 1  |  1  |     0     |           0            |

In [2]:
# Importações e definição da função degrau
import numpy as np

def step_function(x):
    return np.where(x >= 0, 1, 0)

In [3]:
# Classe do Perceptron
class Perceptron:
    def __init__(self, input_size, learning_rate=1.0, epochs=10):
        self.weights = np.zeros(input_size + 1)
        self.learning_rate = learning_rate
        self.epochs = epochs

    def predict(self, x):
        x_with_bias = np.insert(x, 0, 1)
        weighted_sum = np.dot(self.weights, x_with_bias)
        return int(step_function(weighted_sum)[()])

    def fit(self, X, y):
        for _ in range(self.epochs):
            for xi, yi in zip(X, y):
                xi_with_bias = np.insert(xi, 0, 1)
                output = self.predict(xi)
                error = yi - output
                self.weights += self.learning_rate * error * xi_with_bias

In [4]:
# Dados para as portas lógicas
X = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1],
])

y_or = np.array([0, 1, 1, 1])
y_and = np.array([0, 0, 0, 1])
y_xor = np.array([0, 1, 1, 0])

In [5]:
# Treinamento dos perceptrons OR e AND
p_or = Perceptron(input_size=2, learning_rate=0.1, epochs=10)
p_and = Perceptron(input_size=2, learning_rate=0.1, epochs=10)

p_or.fit(X, y_or)
p_and.fit(X, y_and)

In [6]:
# Criando entradas para o Perceptron final (XOR)
intermediate = []
for xi in X:
    or_out = p_or.predict(xi)
    and_out = p_and.predict(xi)
    xor_input = np.array([or_out, 1 - and_out])
    intermediate.append(xor_input)

intermediate = np.array(intermediate)

In [7]:
# Treinamento do perceptron final para XOR
p_xor = Perceptron(input_size=2, learning_rate=0.1, epochs=10)
p_xor.fit(intermediate, y_xor)

In [8]:
# Testando o sistema encadeado
print("Teste do sistema composto de Perceptrons para XOR:")
for xi in X:
    or_out = p_or.predict(xi)
    and_out = p_and.predict(xi)
    xor_input = np.array([or_out, 1 - and_out])
    xor_out = p_xor.predict(xor_input)
    print(f"Entrada: {xi}, Saída XOR prevista: {xor_out}")

Teste do sistema composto de Perceptrons para XOR:
Entrada: [0 0], Saída XOR prevista: 0
Entrada: [0 1], Saída XOR prevista: 1
Entrada: [1 0], Saída XOR prevista: 1
Entrada: [1 1], Saída XOR prevista: 0
